# Image Features

## Goals:
- learn feature extraction, first step of use image features for applications
- learn what characteristics make good image features
- learn different algorithms used to extract features in images

## 1. Feature Dectecion
- features are points of interest in an image
- points of interest should have the following characteristics:
    - Saliency: distinctive, identifiable, and different from its imemediate neighborhood
    - Reppeatability: can be found in multiple images using same operations
    - Locality: occupies a relatively small subset of image space
        - should not change much under small changes in image
    - Quantity: enough points represented in the image
    - Efficency: reasonable computing time as a preprocessing step


Extraction:

- repetitive texture less patches are challenging to detect consistently
- patches with large contrast changes (gradients) are easier to detect (edges)
- gradients in at least two (significantly) different orientations are the easiest to detect (corners)
    - famous corner algorithm: Harris Corner Detection


### 1.1 Feature Detection Algorithms
- Harris -> corners
  - easy to compute, but not sacale invariant
- Harris-Laplace -> corners
  - scale invariant
- Features from acclerated segment test (FAST) -> corners
  - machine learning approach for fast corner detection
- Laplacian of Gaussian (LOG) detector -> blobs
  - scale invariant, but not rotation invariant
- Difference of Gaussian (DOG) detector -> blobs
  - approximate LOG, but faster


### 1.1 Harris Corner Detection


## 2. Feature Descriptors

- what characteristics make a good feature descriptor
- different algorithms used to extract feature descriptors from images


- Feature: points of interest in an image defined by pixel coordinates $[u,v]$
- Descriptor: an N-dimensional vector that provides a summary of the image information around the detected feature $\{f_1, f_2, ..., f_N\}$

- Feature descriptors should have the following characteristics:
    - Distinctiveness: different features should have different descriptors
    - Robustness: descriptors should be able to be matched even if the image is transformed
    - Compactness: descriptors should be small enough to be stored and compared efficiently
    - Efficiency: descriptors should be able to be computed quickly


SIFT: Scale Invariant Feature Transform
1. 16X16 pixel window around detected feature -> feature local neighborhood
2. separate into 4x4 cells
3. compute edge orientation of each pixel in the cell
4. suppress weak edges using predefined threshold
5. construct 32 dimensional histogram of edge orientations for each cell, then concatenate to form 128 dimensional feature descriptor

- SIFT is used in many state-of-art systems
- Combined with DOG feature detector, SIFT descriptors provide a scalue, rotation, and illumation invariant feature detector/descriptor pair


Other Feature Descriptors:
- Speeded Up Robust Features (SURF)
- Gradient Location and Orientation Histogram (GLOH)
- Binary Robust Independent Elementary Features (BRIEF)
- Oriented FAST and Rotated BRIEF (ORB)
